In [1]:
from datetime import datetime, timedelta
import pickle
import os
import pandas as pd
import numpy as np
from tqdm import tqdm
import re

from tldextract import extract
import langid

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 50)

from newspaper import Article

<ipython-input-1-6e87932399ef>:12: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('display.max_colwidth', -1)


NOTE: Mediacloud news for state-specific collection includes news ABOUT the particular state, and as such, might sometimes contain news from national outlets where the article itself might be focused on the state itself. Most (if not all) of the collection is going to be sourced from local state-specific outlets, but not always. Currently, we are keeping all URLs within the obtained collection for the state, except those from nytimes and foxnews since those are our chosen national outlets. Trusting mediacloud's collections to be truly state-focused even when urls come from national outlets. 

In [2]:
start_date = datetime.strptime('2023-04-01', '%Y-%m-%d').date()
end_date = datetime.strptime('2023-06-30', '%Y-%m-%d').date()

In [3]:
cali_news1 = pd.read_csv('data/mediacloud_california_apr2023.csv')
cali_news2 = pd.read_csv('data/mediacloud_california_may2023.csv')
cali_news3 = pd.read_csv('data/mediacloud_california_jun2023.csv')
cali_news = pd.concat([cali_news1, cali_news2, cali_news3], 0)
print(cali_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26084 entries, 0 to 9690
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ap_syndicated         26084 non-null  bool   
 1   collect_date          26084 non-null  object 
 2   feeds                 0 non-null      float64
 3   guid                  26084 non-null  object 
 4   language              26084 non-null  object 
 5   media_id              26084 non-null  int64  
 6   media_name            26084 non-null  object 
 7   media_url             26084 non-null  object 
 8   metadata              26084 non-null  object 
 9   processed_stories_id  26084 non-null  int64  
 10  publish_date          26084 non-null  object 
 11  stories_id            26084 non-null  int64  
 12  story_tags            26084 non-null  object 
 13  title                 26084 non-null  object 
 14  url                   26084 non-null  object 
 15  word_count          

<ipython-input-3-b0165068f754>:4: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only.
  cali_news = pd.concat([cali_news1, cali_news2, cali_news3], 0)


In [4]:
cali_news[['language', 'media_name', 'media_url', 'publish_date', 'title', 'url']].tail(20)

,language,media_name,media_url,publish_date,title,url
9671,en,Democracy Now!,http://www.democracynow.org/,2023-06-30 08:42:00,"Is Supreme Court's ""Gay Wedding"" Case Built on a Lie? Man at Center of the Story Says He's Straight",http://www.democracynow.org/2023/6/30/fake_gay_marriage_website_scotus_case
9672,en,Mendocino Beacon,http://www.mendocinobeacon.com/,2023-06-30 13:43:04,Maximize memory function with a nutrient-rich diet,https://www.mendocinobeacon.com/2023/06/30/maximize-memory-function-with-a-nutrient-rich-diet/
9673,en,Ft. Bragg Advocate-News,http://www.advocate-news.com/,2023-06-30 13:43:04,Maximize memory function with a nutrient-rich diet,https://www.advocate-news.com/2023/06/30/maximize-memory-function-with-a-nutrient-rich-diet/
9674,en,keyt.com,http://www.keyt.com#spider,2023-06-30 22:42:34,Louisiana Democratic governor vetoes bill banning gender-affirming care for minors,https://keyt.com/news/2023/06/30/louisiana-democratic-governor-vetoes-bill-banning-gender-affirming-care-for-minors/
9675,en,San Diego Union Tribune,http://sandiegouniontribune.com/,2023-06-30 00:28:44,Trump and DeSantis court Moms for Liberty in a sign of the group's rising influence over the GOP,https://www.sandiegouniontribune.com/news/nation-world/story/2023-06-29/trump-desantis-among-2024-gop-hopefuls-set-to-appear-at-moms-for-liberty-gathering
9676,en,SFist: San Francisco,http://sfist.com,2023-06-30 18:46:04,"San Francisco Has A Sheriff’s Deputy Who’s Also A Drag Queen, WooWoo Monroe",https://sfist.com/2023/06/30/san-francisco-has-a-sheriffs-deputy-whos-also-a-drag-queen-woowoo-monroe/
9677,en,http://www.hoffmania.com/,http://www.hoffmania.com/,2023-06-30 14:02:00,35 LGBTQ+ owned brands to support this year and beyond,https://www.nbcnews.com/select/shopping/lgbtq-owned-business-ncna1272731
9678,en,knzr.com,http://knzr.com/,2023-06-30 18:02:47,Trans Influincer Lashes Out At Beer Brewer,https://www.knzr.com/trans-influincer-lashes-out-at-beer-brewer/
9679,en,Perris Progress,http://www.theperrisprogress.com/,2023-06-30 12:34:48,"FRIDAY June 30, 2023",https://zapinin.com/this-date-in-history/friday-june-30-2023/
9680,en,Nature,http://www.nature.com/,2023-06-30 20:00:00,Architecture of a complete Bce-type antimicrobial peptide resistance module,https://www.nature.com/articles/s41467-023-39678-w


In [5]:
relevant_news = cali_news[~cali_news['media_name'].isin(['Fox News', 'New York Times'])]
relevant_news = relevant_news[relevant_news['language']=='en']
relevant_news = relevant_news[['media_name', 'publish_date', 'title', 'url']]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25794 entries, 2 to 9690
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   media_name    25794 non-null  object
 1   publish_date  25794 non-null  object
 2   title         25794 non-null  object
 3   url           25794 non-null  object
dtypes: object(4)
memory usage: 1007.6+ KB
None


In [6]:
relevant_news.publish_date = pd.to_datetime(relevant_news.publish_date)
relevant_news = relevant_news[relevant_news['publish_date'].dt.date >= start_date]
relevant_news = relevant_news[relevant_news['publish_date'].dt.date <= end_date]
print(relevant_news.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25794 entries, 2 to 9690
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    25794 non-null  object        
 1   publish_date  25794 non-null  datetime64[ns]
 2   title         25794 non-null  object        
 3   url           25794 non-null  object        
dtypes: datetime64[ns](1), object(3)
memory usage: 1007.6+ KB
None


In [7]:
def get_article_from_url(url):
    try:
        article = Article(url)
        article.download()
        article.parse()
        subtitle = article.meta_description
        if subtitle == '' or subtitle == ' ':
            subtitle = None
        text = article.text
        if text == '' or text == ' ':
            text = None
    except:
        subtitle = None
        text = None
    return subtitle, text

In [8]:
subtitles, texts = [], []
all_urls = list(relevant_news['url'])
for url in tqdm(all_urls):
    subtitle, text = get_article_from_url(url)
    subtitles.append(subtitle)
    texts.append(text)
relevant_news['subtitle'] = subtitles
relevant_news['text'] = texts
print(relevant_news.info())

  0%|          | 0/25794 [00:00<?, ?it/s]/home/pranavgoel/miniconda3/envs/pg3/lib/python3.8/site-packages/dateutil/parser/_parser.py:1207: UnknownTimezoneWarning: tzname PDT identified but not understood.  Pass `tzinfos` argument in order to correctly return a timezone-aware datetime.  In a future version, this will raise an exception.
  warnings.warn("tzname {tzname} identified but not understood.  "
100%|██████████| 25794/25794 [3:40:11<00:00,  1.95it/s]   

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25794 entries, 2 to 9690
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   media_name    25794 non-null  object        
 1   publish_date  25794 non-null  datetime64[ns]
 2   title         25794 non-null  object        
 3   url           25794 non-null  object        
 4   subtitle      22522 non-null  object        
 5   text          24285 non-null  object        
dtypes: datetime64[ns](1), object(5)
memory usage: 1.4+ MB
None


In [9]:
relevant_news.to_csv('data/california_article_texts_and_info.csv',
                     index=False)